In [33]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.fullDataExtraction import extarctChannelNames
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math
from tensorflow import keras
import dash_bootstrap_components as dbc
from ipynb.fs.full.fullDataExtraction import compute_diffEnt
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate
import dash
import dash_mantine_components as dmc
from dash_iconify import DashIconify

In [34]:
def makeEpochs(rawArray):
    return mne.make_fixed_length_epochs(rawArray, duration=1, preload=True,verbose=0)

def getRawArrays(matfile):
    del matfile["__header__"]
    del matfile["__version__"]
    del matfile["__globals__"]
    clip_info={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    keyName=list(matfile.keys())[0][:-1]
    channelNamesExtraction=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    info=mne.create_info(channelNamesExtraction,200,'eeg')
    for ind,i in enumerate(indexs):
        rawData=matfile[keyName + str(i)]
        clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)
    return clip_info

def ExtractFeatures(sliced_epochs,features):
    combined = np.zeros((1,len(features)*62))
    epoch_array=mne.Epochs.get_data(sliced_epochs)
    extracted_data=extract_features(epoch_array,200,features)
    combined = np.vstack((combined,extracted_data))
    combined = np.delete(combined, 0, axis=0)
    return combined

def extend200(arr):
    temp=[]
    for i in arr:
        for j in range(0,200):
            temp.append(i)
    return temp

In [35]:
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")
channelNames=channels.iloc[:,0]
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
channelNames.insert(0,"Fp1")
ch_types = ['eeg'] * len(channelNames)

info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)
info.set_montage('standard_1020')

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF3, AF4, F7, F5, F3, F1, Fz, F2, F4, F6, F8, ...
 chs: 60 EEG
 custom_ref_applied: False
 dig: 63 items (3 Cardinal, 60 EEG)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 60
 projs: []
 sfreq: 200.0 Hz
>

In [36]:
def deleteUselessNodes(data):
    data=np.delete(data, 61, 0)
    data=np.delete(data, 57, 0)
    return data

In [37]:
model = keras.models.load_model("emotionPredictionModel")
subject=loadmat('data/subject.mat')
rawArrays=getRawArrays(subject)
keyName=list(subject.keys())[3][:-1]
data=[]
evoked=[]
df=[]
y_pred=[]
extractedData=[]
transData=[]
for i in range(1,16):
    data.append(deleteUselessNodes(subject[keyName+str(i)]))
    evoked.append(mne.EvokedArray(data[i-1],info))
    transData.append(np.transpose(data[i-1]))
    df.append(pd.DataFrame(transData[i-1], columns = channelNames))
    extractedData.append(ExtractFeatures(makeEpochs(rawArrays[i-1]),['hjorth_mobility', ('diffEnt',compute_diffEnt), 'rms', 'skewness']))
    prediction= model.predict(extractedData[i-1])
    y_pred.append(extend200((prediction > 0.5).astype(int).tolist()))

7/7 [==============================] - 0s 3ms/step


In [38]:
frontTup=("Frontal",['Fp1', 'Fpz', 'Fp2','AF3', 'AF4','F1', 'Fz', 'F2'])
centralTup=("Central",['FC1', 'FCz', 'FC2','C3', 'C1', 'Cz', 'C2', 'C4'])
parietalTup=("Parietal",['CP3', 'CP1', 'CPz', 'CP2', 'CP4','P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6'])
occipitalTup=("Occipital",['PO7', 'PO5', 'PO3','POz', 'PO4', 'PO6', 'PO8','O1', 'Oz', 'O2'])
leftTup=("LeftTemporal",['F4', 'F6', 'F8','FC4','FC6', 'FT8','C6','CP6', 'TP8','P8'])
rightTup=("RightTemporal",['F7', 'F5', 'F3','FT7', 'FC5', 'FC3','C5','TP7', 'CP5','P7'])
allSecList=[frontTup,centralTup,parietalTup,occipitalTup,leftTup,rightTup]

## Main component definition

### slider/input

In [39]:
choice = 0

## Slider
slider = html.Div(
    [
            dcc.Slider(
            min=0,
            max=data[choice].shape[1]-1,
            step=1,
            id="time",
            marks=None,
            tooltip={"always_visible": False},
            
     
            ),
      
    ],
    
    style={'padding-top':'2rem','color':"teal" }
)


#sliderLabel=dbc.Row(html.Label("s"),style ={"text-align": "center"})

#sliderColumn= dbc.Col([slider],width=2,style = {"height":"30%","text-align": "center",'padding-left':'50%', 'padding-right':'50%'})


####################   Time Selection   #################### 

# Label 
inputLabel=html.Div([
    html.H3("Select time frame")
])

inputCol=dbc.Col(
    dcc.Input(
        id="inputTime",
        type="number",
        placeholder="Select time",
        min=0,
        max=data[choice].shape[1]-1,
        style={'vertical align':'top'}
        
        ),
    style={'padding-top':'3rem','vertical-align': 'top','text-align': 'center'},
    width=6
)


inputField=html.Div([
    
    dbc.Row([
        dbc.Col(inputLabel),
        inputCol,
        
    ]),
    dbc.Row([
        slider,
       ])
    
], style={'height':'30vh','backgroundColor': '#18242B'}
)

## drop down for node selection

In [40]:
def buildOneDdRow(options1,options2,options3):
    return dbc.Row([
    dbc.Col([
        html.B(html.P(options1[0])),
        dcc.Dropdown(
            options1[1],
            multi=True,
            id=options1[0]
        )
    ],
        width=4
    ),
    dbc.Col([
        html.B(html.P(options2[0])),
        dcc.Dropdown(
            options2[1],
            multi=True,
            id=options2[0]
        )
    ],
        width=4
    ),
    dbc.Col([
        html.B(html.P(options3[0])),
        dcc.Dropdown(
            options3[1],
            multi=True,
            id=options3[0]
        )
    ],
        width=4
    )
],
    style={'padding-top':'3rem', 'padding-left':'3rem'})

dropDownLabel=html.Div([
    html.H3("Node selection"),
    
],
    style={'padding-top':'2rem'}
)

dropDownRow = (dbc.Row([dropDownLabel]),
dbc.Row([
                    buildOneDdRow(frontTup,centralTup,parietalTup,),
                    buildOneDdRow(occipitalTup,leftTup,rightTup)
              ])
              )
        
   

### Advanced option selection

In [41]:
advancedOptionRow=dbc.Row(
    dmc.Switch(
        id="moreOptionsChecklist", 
        label="Select For More Options",
        style={'color': '#95969A'},
        persistence_type='local',
        checked=False,
        color="teal"
    ),
    id='moreOptionsSel'
)

### clip select

In [42]:
clipSelectLabel=html.Div([
    html.H3(["select clip"])
])

clipLabels=[]

for i in range(1,16):
    clipLabels.append(str(i))

clipSelectDD=dbc.Row([
    dbc.Col(
        clipSelectLabel,
        width=6
    ),
    dbc.Col(
        dcc.Dropdown(
            clipLabels,
            placeholder="1",
            id="clipDrop"
        ),
        
    ),
    ],
    style={'padding-bottom':'2rem'}
)

clipField=html.Div([
    
    clipSelectDD,
    advancedOptionRow,
    dbc.Row(id="moreOptions")
    
], style={'margin-top':'2rem',
          'backgroundColor': '#18242B',
          'overflow-y':'scroll',
          'overflow-x':'hidden'
           },
)

## value pick tab

In [43]:
optionStack=dbc.Col(
                [
                    dbc.Stack([ 
                        inputField,
                        clipField,
                        #slider,
                        #clipSelectDD,
                        #advancedOptionRow,
                        #dbc.Row(id="moreOptions")
                    ])
                ],
                width=6,
                style={
                    'display':'flex', 
                    'justifyContent':'center',
                    'padding-top':'2rem',
                    'padding-right':'2rem',
                    'padding-left':'2rem',
                    'height':'90vh',
                    'overflow-y':'hidden',
                    'overflow-x':'hidden'
                },
                id='optionPick'
            )

emotionCol=dbc.Col(
                id="emotion",
                style={'padding-top':'1rem',
                       "text-align": "center",
                       'border-radius': '0% 0% 0% 0% / 0% 0% 0% 0%',
                       'transition': 'all .4s ease'
                       
                      },
                width=6
            )

valSec=dbc.Row([
            optionStack,
            emotionCol
        ],style={'margin-top':'1rem'})

## charts

In [44]:
titleStyle={'border-bottom': 'thin solid black'}

def buildChartTitle(cType):
    return html.Div(
        cType + " of Current Node Readings",
        style=titleStyle
    )

heatMap=dbc.Col([
    buildChartTitle("Heat Map"),
    dbc.Col(
        id='heatmap',
        style={'padding-bottom':'3rem'}
    )
],
)


#takes in the type of chart and a list of 2-tuples with the format 0: section name 1: channel names(list)
def buildCharts(type,list):
    res=[]
    for i in list:
        res.append(dbc.Row(dbc.Col(id=(type+i[0]))))
    return res

advancedBarCharts=dbc.Col([
    buildChartTitle("Bar Charts"),
    dbc.Col(
        buildCharts('barChart',allSecList),
        id="barChartContainer",
        style={
            'height':'65vh',
            'overflow-x': 'hidden',
            'overflow-y': 'hidden'
        
            
        }
    )
],
)

advancedLineCharts=dbc.Row([
    buildChartTitle("Global field power (GFP)"),
    dbc.Col(
        buildCharts('lineChart',allSecList),
        id="lineChartContainer",
        style={
            'height':'65vh',
            'overflow-x': 'hidden',
            'overflow-y': 'hidden'
            
        },
        width=6
    ),
    dbc.Col(
        id="GFPContainer",
        width=6
    )
])

## charts section

### Heatmap section

In [45]:
heatMapSec=dbc.Row([
    dbc.Col(
        [
            heatMap,
        ],
        id='heatMapDisplay',
        width=8
    ),
    dbc.Col(
        [
        ],
        id='heatMapDes',
        width=4
    )
],
    style={'padding-top':'2rem'}
)

### Bar Chart Section

In [46]:
barChartSec=dbc.Row([
    dbc.Col(
        [
            advancedBarCharts,
        ],
        id='barChartDisplay',
        width=8
    ),
    dbc.Col(
        [
        ],
        id='barChartDes',
        width=4
    )
],
    style={'padding-top':'2rem'}
)

### GFP section

In [47]:
GFPSec=dbc.Row([
    
            advancedLineCharts
        ],
            id='GFPDisplay'
    
),
dbc.Row([],
     
         
         id='GFPDes',  
         style = {'padding-top':'2rem'}
)

Row(children=[], id='GFPDes', style={'padding-top': '2rem'})

### Helper functions

In [48]:
def decipherEmotion(time_instance,ddChoice):
    
    if time_instance != None:
        
        if (y_pred[ddChoice][time_instance] == [1,0,0]):
            emotion="positive",
            row = html.Div([
                    
                    html.Img(src="assets/positive.png"),
                    html.H4(["You are currently feeling happy!"]),
                           "Here are some activities you can do: "
                
                ])      
             
            
        elif (y_pred[ddChoice][time_instance] == [0,1,0]):
            emotion="neutral"
            row = html.Div([
                    
                    html.Img(src="assets/neutral.png"),
                    html.H4(["You are currently feeling neutral"]),
                           "Here are some activities you can do: "
                   ])  
            
        else:
            emotion="negative" 
            row = html.Div([
                    
                    html.Img(src="assets/negative.png"),
                    html.H4(["You are currently feeling sad"]),
                           "Here are some activities you can do: "
                
                ])  
            
           
        emotionDiv = html.H2(emotion)
        
        
        return emotionDiv, row 
    
    else:
        return html.H3("No time frame has been selected yet")
    
def makeValueMatrix(currentAnalysed):
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    #tracker value
    k=0
    for i in range(0,result.shape[0]):
        for j in range (0,result.shape[1]):
            if not(math.isnan(result[i][j])):
                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result

def returnEmptyGraph():
    fig = go.Figure(go.Scatter(x=[], y = []))
    fig.update_layout(template = None) #, plot_bgcolor='#18242B',paper_bgcolor='#18242B',font_color='#95969A')
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)
    empty=dcc.Graph(figure=fig)
    return empty

def getAxisNames():
    return dict(x="Channel Names", y="Voltage", color="Channel Names")

### Layout

In [49]:
external_stylesheets = ["assets/style.css", dbc.themes.LUX]
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

app.layout = dbc.Container([
    
    dcc.Store(id='timeStore'),
    dcc.Store(id='clipChoice'),
    dcc.Store(id='frontChoice'),
    dcc.Store(id='centralChoice'),
    dcc.Store(id='parietalChoice'),
    dcc.Store(id='occipitalChoice'),
    dcc.Store(id='leftChoice'),
    dcc.Store(id='rightChoice'),
    dcc.Store(id='advancedChoice'),
    
    dbc.Row([
        dbc.Row(
            html.H1('Emotion Prediction With MNE'),
            style={'padding-top':'1rem','padding-bottom':'1rem'}
        ),
        dbc.Row(
            dmc.Tabs(
                    [
                        dmc.TabsList(
                            
                            
                            [
                                
                                dmc.Tab('VALUES', icon=DashIconify(icon="material-symbols:display-settings-rounded"), value='valSec',style={'color': 'white'}),
                                dmc.Tab('HEAT MAP', icon=DashIconify(icon="mdi:head"), value='chartSec',style={'color': 'white'}),
                                dmc.Tab('BAR CHARTS', icon=DashIconify(icon="ion:bar-chart-sharp"),value='barSec',style={'color': 'white'}),
                                dmc.Tab('GFP ANALYSIS', icon=DashIconify(icon="codicon:graph-line"),value='GFPSec',style={'color': 'white'}),   
                                
                            ],
                            #styling 
                            grow = True,
                            
                            style= {
                            'backgroundColor': '#18242B',
                            'borderColor': '#23262E',
                            'display': 'flex',
                            'flex-direction': 'row',
                            'alignItems': 'center',
                            'justifyContent': 'center'
                        },
                            
                         
                            
                        )                         
                    ],
                    id="tabSel",
                    value='valSec',
                    color="teal"
                )
            ),
        dbc.Row(id='mainContent',children=valSec),
    ]),
])

## Callbacks

### advanced option callbacks

In [50]:
#callback to update advanced options
@app.callback(
    Output("moreOptions", "children"), 
    Input("moreOptionsChecklist", "checked"),
    Input('advancedChoice','data')
)
def showAdvancedOption(checked, savedData):
    print("storedData : ", savedData)
    print("Input : ", checked)
    if checked or savedData:
        return dropDownRow
    else:
        return " "
    
#callback to store the value of the advanced option switch
@app.callback(
    Output('advancedChoice','data'),
    Input("moreOptionsChecklist", "checked"),
)
def storeAdvancedChoice(checked):
    if checked == None or False:
        raise PreventUpdate
    else:
        return checked

#callback to dynamically update the value section scroll bar
@app.callback(
    Output('optionPick','style'),
    Input("moreOptionsChecklist", "checked")
)
def updateScroll(checked):
    if (checked):
        return {
                'display':'flex', 
                'justifyContent':'center',
                'padding-top':'2rem',
                'height':'75vh',
                'overflow-y':'scroll',
                'overflow-x':'hidden'
                }
    else:
        return {
                'display':'flex', 
                'justifyContent':'center',
                'padding-top':'2rem',
                'height':'75vh',
                'overflow-y':'hidden',
                'overflow-x':'hidden'
                }

### dropdown callbacks

In [51]:
#callback to store the clip choice
@app.callback(
    Output('clipChoice','data'),
    Input('clipDrop','value')
)
def filterClip(ddChoice):
    if not ddChoice:
        return 0
    return int(ddChoice)-1

#Clip choice
@app.callback(
    Output('clipDrop','value'),
    Input('clipDrop','value'),
    Input('clipChoice','data')
)
def accessStoredClip(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default

#callback to store frontal node choices
@app.callback(
    Output('frontChoice','data'),
    Input(frontTup[0],'value')
)
def filterFrontal(choices):
    if not choices:
        return frontTup[1]
    return choices

#callback to store central node choices
@app.callback(
    Output('centralChoice','data'),
    Input(centralTup[0],'value')
)
def filterCentral(choices):
    if not choices:
        return centralTup[1]
    return choices

#callback to store parietal node choices
@app.callback(
    Output('parietalChoice','data'),
    Input(parietalTup[0],'value')
)
def filterParietal(choices):
    if not choices:
        return parietalTup[1]
    return choices



#callback to store occipital node choices
@app.callback(
    Output('occipitalChoice','data'),
    Input(occipitalTup[0],'value')
)
def filterOccipital(choices):
    if not choices:
        return occipitalTup[1]
    return choices



#callback to store left node choices
@app.callback(
    Output('leftChoice','data'),
    Input(leftTup[0],'value')
)
def filterLeft(choices):
    if not choices:
        return leftTup[1]
    return choices



#callback to store right node choices
@app.callback(
    Output('rightChoice','data'),
    Input(rightTup[0],'value')
)
def filterRight(choices):
    if not choices:
        return rightTup[1]
    return choices




#Frontal channel choices
@app.callback(
    Output(frontTup[0],'value'),
    Input(frontTup[0],'value'),
    Input('frontChoice','data')
)
def accessStoredFrontal(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default

#Central channel choices
@app.callback(
    Output(centralTup[0],'value'),
    Input(centralTup[0],'value'),
    Input('centralChoice','data')
)
def accessStoredCentral(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
#Parietal channel choice
@app.callback(
    Output(parietalTup[0],'value'),
    Input(parietalTup[0],'value'),
    Input('parietalChoice','data')
)
def accessStoredParietal(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
#Occipital channel choice
@app.callback(
    Output(occipitalTup[0],'value'),
    Input(occipitalTup[0],'value'),
    Input('occipitalChoice','data')
)
def accessStoredOccipital(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default

#left channel choice
@app.callback(
    Output(leftTup[0],'value'),
    Input(leftTup[0],'value'),
    Input('leftChoice','data')
)
def accessStoredLeft(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
#Right channel choice
@app.callback(
    Output(rightTup[0],'value'),
    Input(rightTup[0],'value'),
    Input('rightChoice','data')
)
def accessStoredRight(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default

### slider/input callbacks

In [52]:
#callback to update slider/input max
@app.callback(
    Output('time','max'),
    Output('inputTime','max'),
    Input('clipDrop','value')
)
def updateMax(ddChoice):
    if not ddChoice:
        return transData[0].shape[0],transData[0].shape[0]
    return transData[int(ddChoice)-1].shape[0],transData[int(ddChoice)-1].shape[0]

#callback to store time
@app.callback(
    Output('timeStore','data'),
    Input('inputTime','value')
)
def storeTime(data):
    if data == None:
        raise PreventUpdate
    else:
        return data
    
#callback to update the slider and input
@app.callback(
    Output("inputTime","value"),
    Output("time","value"),
    Input("inputTime","value"),
    Input("time","value"),
    Input("timeStore","data")
)
def updateSldierOrInput(inputVal,sliderVal,timeStore):
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]["prop_id"].split(".")[0]
    value1 = inputVal if trigger_id == "inputTime" else sliderVal
    value2 = timeStore
    if value1 == None:
        if value2 != None:
            return value2, value2
    else:
        return value1, value1

### tab callbacks

In [53]:
#callback to render out the tabs
@app.callback(
    Output('mainContent','children'),
    Input('tabSel','value')
)
def renderContent(tab):
    if tab=='valSec':
        return valSec
    elif tab == 'barSec':
        return barChartSec
    elif tab == 'GFPSec':
        return GFPSec
    else:
        return heatMapSec
    

### emotion display callback

In [54]:
#callback to update the emotions display
@app.callback(
    Output('emotion','children'),
    Input('inputTime','value'),
    Input('timeStore','data'),
    Input('clipDrop','value')
)
def updateEmotions(curTime,storedTime,ddChoice):
    if (curTime!= None) and (ddChoice != None):
        return decipherEmotion(curTime,int(ddChoice)-1)
    else:
        return decipherEmotion(storedTime,0)

### chart callbacks

In [55]:
#callback to update heatmap
@app.callback(
    Output('heatmap','children'),
    Input('timeStore','data'),
    Input('clipChoice','data'),
)
def updateHeatMap(time_instance,ddChoice):
    if (time_instance is None):
        raise PreventUpdate
    else:
        axis_names=getAxisNames()
        
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
            
        heatMap = px.imshow(makeValueMatrix(transData[ddChoice][time_instance]))
        heatMap.update_xaxes(showticklabels=False)
        heatMap.update_yaxes(showticklabels=False)
        heatMap.update_layout(xaxis=dict(scaleanchor='y', constrain='domain'),
                              coloraxis=dict(colorbar=dict(orientation='h',thickness=20,len=1.0)),
                              plot_bgcolor='#18242B',
                              paper_bgcolor='#18242B',
                              font_color='#95969A')
        heatMapFig=dcc.Graph(figure=heatMap)
        
        return heatMapFig

#callback to update bar charts
@app.callback(
    Output("barChartFrontal", "children"),
    Output("barChartCentral", "children"),
    Output("barChartParietal", "children"),
    Output("barChartOccipital", "children"),
    Output("barChartLeftTemporal", "children"),
    Output("barChartRightTemporal", "children"),
    Output("barChartContainer","style"),
    Input('timeStore','data'),
    Input('frontChoice','data'),
    Input('centralChoice','data'),
    Input('parietalChoice','data'),
    Input('occipitalChoice','data'),
    Input('leftChoice','data'),
    Input('rightChoice','data'),
    Input('clipChoice','data'),
    Input("advancedChoice", "data")
)
def updateChartSec(time_instance,front,central,parietal,occipital,left,right,ddChoice,checked):
    if (time_instance is None):
        raise PreventUpdate
    else:
        axis_names=getAxisNames()
        
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
        
        #indicates that the advanced option is not selected
        if not checked:
            ser=currentDf[frontTup[1]].iloc[time_instance]
            oneBar=px.bar(x=ser.axes[0].tolist(),y=ser.values,color=ser.axes[0].tolist())
            oneBar.update_layout(xaxis_title=axis_names['x'],
                                 yaxis_title=axis_names['y'],
                                 legend_title=axis_names['color'],
                                 plot_bgcolor='#18242B',
                                 paper_bgcolor='#18242B',
                                 font_color='#95969A'
                                )
            oneBarGraph=dcc.Graph(figure=oneBar)
            style={'height':'65vh','overflow-x': 'hidden','overflow-y': 'hidden'}
            return oneBarGraph,returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),style
        
        
        
        ser=currentDf[front].iloc[time_instance]
        frontalBar=px.bar(x=front,y=ser.values,color=front,title="Frontal",labels=axis_names)
        frontalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartFrontal= dcc.Graph(figure=frontalBar)
        
    
        ser=currentDf[central].iloc[time_instance]
        centralBar=px.bar(x=central,y=ser.values,color=central,title="Central",labels=axis_names)
        centralBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        barChartCentral= dcc.Graph(figure=centralBar)
        
        
        ser=currentDf[parietal].iloc[time_instance]
        parietalBar=px.bar(x=parietal,y=ser.values,color=parietal,title="Parietal",labels=axis_names)
        parietalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartParietal= dcc.Graph(figure=parietalBar)
        
        
        ser=currentDf[occipital].iloc[time_instance]
        occipitalBar=px.bar(x=occipital,y=ser.values,color=occipital,title="Occipital",labels=axis_names)
        occipitalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartOccipital= dcc.Graph(figure=occipitalBar)
    
    
    
        ser=currentDf[left].iloc[time_instance]
        ltBar=px.bar(x=left,y=ser.values,color=left,title="Left Temporal",labels=axis_names)
        ltBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartLeftTemporal= dcc.Graph(figure=ltBar)
    
    
    
        ser=currentDf[right].iloc[time_instance]
        rfBar=px.bar(x=right,y=ser.values,color=right,title="Right Temporal",labels=axis_names)
        rfBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        barChartRightFrontal= dcc.Graph(figure=rfBar)
        
        style={'height':'65vh','overflow-x': 'hidden','overflow-y': 'scroll'}
        
        return barChartFrontal,barChartCentral,barChartParietal,barChartOccipital,barChartLeftTemporal,barChartRightFrontal,style
    
#callback to update the GFP/line charts
@app.callback(
    Output("lineChartFrontal", "children"),
    Output("lineChartCentral", "children"),
    Output("lineChartParietal", "children"),
    Output("lineChartOccipital", "children"),
    Output("lineChartLeftTemporal", "children"),
    Output("lineChartRightTemporal", "children"),
    Output("GFPContainer","children"),
    Output("lineChartContainer","style"),
    Input('timeStore','data'),
    Input('frontChoice','data'),
    Input('centralChoice','data'),
    Input('parietalChoice','data'),
    Input('occipitalChoice','data'),
    Input('leftChoice','data'),
    Input('rightChoice','data'),
    Input('clipChoice','data'),
    Input("advancedChoice", "data")
)
def outPutGFPCharts(time_instance,front,central,parietal,occipital,left,right,ddChoice,checked):
    axis_names=getAxisNames()
    if (time_instance is None):
        raise PreventUpdate
    else:
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
            
        maxTf=time_instance+100
        minTf=time_instance-100
        
        if minTf<0:
            minTf=0
        if maxTf>=data[ddChoice].shape[1]:
            maxTf=data[ddChoice].shape[1]-1
        
        GFP=evoked[ddChoice].data.std(axis=0, ddof=0)[minTf:maxTf]
        GFPfigure=px.line(GFP)
        GFPfigure.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        GFPFig=dcc.Graph(figure=GFPfigure)
        
        if not checked:
            ser=currentDf[frontTup[1]].loc[minTf:maxTf:1]
            oneLine=px.line(ser,labels=axis_names)
            oneLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            singleGFP = dcc.Graph(figure=oneLine)
            
            style={'height':'65vh','overflow-x': 'hidden','overflow-y': 'hidden'}
            
            return singleGFP,returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),GFPFig,style
        
        else:
            frontalLine=px.line(currentDf[front].loc[minTf:maxTf:1])
            frontalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartFrontal= dcc.Graph(figure=frontalLine)
            
            
            centralLine=px.line(currentDf[central].loc[minTf:maxTf:1])
            centralLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartCentral= dcc.Graph(figure=centralLine)
    
            parietalLine=px.line(currentDf[central].loc[minTf:maxTf:1])
            parietalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartParietal= dcc.Graph(figure=parietalLine)
            
            
            occipitalLine=px.line(currentDf[occipital].loc[minTf:maxTf:1])
            occipitalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartOccipital= dcc.Graph(figure=occipitalLine)
            
            
            ltLine=px.line(currentDf[left].loc[minTf:maxTf:1])
            ltLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartLeftTemporal= dcc.Graph(figure=ltLine)
    
            
            rtLine=px.line(currentDf[right].loc[minTf:maxTf:1])
            rtLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartRightFrontal= dcc.Graph(figure=rtLine)
        
            style={'height':'65vh','overflow-x': 'hidden','overflow-y': 'scroll'}
            
            return lineChartFrontal,lineChartCentral,lineChartParietal,lineChartOccipital,lineChartLeftTemporal,lineChartRightFrontal,GFPFig,style
    
            

### Run Server

In [56]:
if __name__ == '__main__':
    app.run_server(port=4400)

Dash app running on http://127.0.0.1:4400/
storedData :  False
Input :  False
storedData :  True
Input :  True
storedData :  False
Input :  False
storedData :  True
Input :  True
storedData :  False
Input :  False
